In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Final_data.csv')

In [3]:
df.head()

,Date,Customer_Name,Item_Name,Vrh_No,Quantity,Price_per_unit,Amount
0,2019-01-04,customer1,Item_1,1,200.0,20.0,4000.0
1,2019-01-04,customer1,Item_2,1,160.0,28.0,4480.0
2,2019-01-04,customer1,Item_3,1,12.0,60.0,720.0
3,2019-01-04,customer1,Item_3,1,15.0,35.0,525.0
4,2019-01-04,customer1,Item_3,1,25.0,25.0,625.0


In [4]:
df['Customer_Name']=df['Customer_Name'].apply(lambda x:x.lstrip('customer'))
df['Item_Name']=df['Item_Name'].apply(lambda x:x.lstrip('Item_'))

In [5]:
df.head()

,Date,Customer_Name,Item_Name,Vrh_No,Quantity,Price_per_unit,Amount
0,2019-01-04,1,1,1,200.0,20.0,4000.0
1,2019-01-04,1,2,1,160.0,28.0,4480.0
2,2019-01-04,1,3,1,12.0,60.0,720.0
3,2019-01-04,1,3,1,15.0,35.0,525.0
4,2019-01-04,1,3,1,25.0,25.0,625.0


In [6]:
df1 = pd.DataFrame()
df1[['Customer_Name','Item_Name']] = df[['Customer_Name','Item_Name']]

In [7]:
data = pd.melt(df1,id_vars=['Customer_Name'], value_name='Item_Name').groupby(['Customer_Name', 'Item_Name'])\
     .agg({'Item_Name': 'count'}).rename(columns={'Item_Name': 'Purchase'})\
     .reset_index().rename(columns={'Item_Name': 'Item_Id','Customer_Name':'Customer'})

In [8]:
data['Purchase'] = data['Purchase'].astype(int)

In [9]:
data

,Customer,Item_Id,Purchase
0,1,1,2
1,1,2,1
2,1,3,5
3,1,4,3
4,1,7,2
...,...,...,...
851,99,3,3
852,99,30,1
853,99,5,1
854,99,52,1


In [10]:
data.columns

Index(['Customer', 'Item_Id', 'Purchase'], dtype='object')

In [11]:
data['Item_Id'] = data['Item_Id'].astype(int)

In [12]:
customer_item_matrix = data.pivot_table(index='Customer',columns='Item_Id',values='Purchase',fill_value=0)

In [13]:
X = customer_item_matrix.T
X

Customer,1,10,100,101,102,103,104,105,106,107,...,90,91,92,93,94,95,96,97,98,99
Item_Id,,,,,,,,,,,,,,,,,,,,,
1,2,1,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,1
2,1,1,0,2,1,4,0,0,0,0,...,0,0,0,0,0,0,2,2,4,0
3,5,6,0,0,0,11,0,0,0,0,...,0,14,0,0,0,2,0,2,7,3
4,3,6,1,0,0,0,0,1,0,0,...,6,0,2,4,2,0,1,5,7,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
74,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75,0,0,0,0,0,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
customer_item_matrix.shape

(141, 77)

In [15]:
from sklearn.decomposition import TruncatedSVD

In [16]:
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(77, 10)

In [17]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(77, 77)

In [18]:
customer_names = list(X.index)

In [19]:
prod=1
customer_names = list(X.index)
customer_ID = customer_names.index(prod)
correlation_product_ID = correlation_matrix[customer_ID]
print(correlation_product_ID)
Recommend = list(X.index[correlation_product_ID > 0.50])
Recommend.remove(prod) 

Recommend[0:6]

[ 1.          0.59957368  0.44200061  0.45892803  0.33908254  0.43901948
  0.51093323  0.54685217  0.36862085  0.73303906  0.13858868  0.37153212
  0.06699194  0.23590322  0.54028863  0.2774907   0.0831999   0.36963403
  0.54763224  0.6302155   0.758242    0.68322146  0.34549721  0.33253138
  0.40990693  0.35082778  0.38077722  0.60061449  0.27495174  0.67961311
  0.37444158  0.4771534   0.64470504  0.42043853  0.30867783  0.50205572
  0.19514342  0.50737151  0.09637778  0.4444858   0.04834098  0.54625407
  0.12603733  0.33200985  0.00312091  0.3508475   0.0947145   0.35959615
 -0.03612521  0.31692434  0.02930511  0.05894378  0.58560015  0.6510596
  0.06211013  0.00119046  0.5944862   0.41598363  0.25739828  0.06933654
  0.31273555  0.31273555  0.31273555  0.31273555  0.31273555  0.31273555
  0.31273555  0.31273555  0.31273555  0.31273555  0.31273555  0.31273555
  0.31273555  0.54663097  0.0572558   0.31273555  0.17166138]


[2, 7, 8, 10, 15, 19]

In [20]:
def recommender(id):
    if id>0 and id<78:
        customer_names = list(X.index)
        customer_ID = customer_names.index(id)
        correlation_product_ID = correlation_matrix[customer_ID]
        correlation_product_ID
        Recommend = list(X.index[correlation_product_ID > 0.50])
        Recommend.remove(id) 
        Recommend=Recommend[0:5]
        print("\n ------SIMILAR ITEMS------- ")
        Recommend=['Item_'+ str(k) for k in Recommend]
        for i in Recommend:
            print(i)
        
    else:
        print("Invalid Item Id")

In [26]:
Item_Id= int(input("Enter Item Id to see similar items:"))

recommender(Item_Id)

Enter Item Id to see similar items:56

 ------SIMILAR ITEMS------- 
Item_3
Item_23
Item_26
Item_37
Item_38
